In [1]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dropout, Flatten, Conv2D, Dense
from sklearn.preprocessing import MinMaxScaler as MMS, StandardScaler as SDS
from tensorflow.keras.callbacks import EarlyStopping 
import numpy as np

# 1. 데이터

In [2]:
dataset = load_boston()        
x = dataset.data                
y = dataset.target              

print("원본 데이터")
# print("x: ", x, "\ny: ", y)
print(x.shape, y.shape)

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=3333)

# scaler = MMS()
scaler = SDS()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

print("split + scailing 데이터")
# print("x_test: ", x_test, "\nx_trian: ", x_train)
# print("y_test: ", y_test, "\ny_trian: ", y_train)
print(x_train.shape, x_test.shape)

# ---------- CNN 모델에 적용해보기 위해 4차원으로 변환 ----------- #
x_train = x_train.reshape(354, 13, 1, 1)
x_test = x_test.reshape(152, 13, 1, 1)


원본 데이터
(506, 13) (506,)
split + scailing 데이터
(354, 13) (152, 13)


c:\Users\bitcamp\anaconda3\envs\tf27\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housin

# 2. 모델


In [3]:
model = Sequential()
model.add(Conv2D(32, (2,1), input_shape = (13, 1, 1), activation='relu'))
model.add(Dropout(0.5)) # 과적합 방지
model.add(Conv2D(16, (2,1), activation='relu'))
model.add(Dropout(0.3)) # 과적합 방지
model.add(Flatten())    # DNN모델에 적용하기 위해 2차원으로 변환
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2)) # 과적합 방지
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

# 3. 컴파일 및 훈련

In [4]:
model.compile(loss='mse', optimizer='adam') # one-hot encoding 하지 않아도 되는 데이터이므로 loss= sparse_categorical_crossentropy

ES = EarlyStopping(monitor='val_loss', mode='auto', patience=4, restore_best_weights=True) 
model.fit(x_train, y_train, epochs=64, batch_size=5, validation_split=0.2, callbacks = [ES], verbose=2) # verbose: 함수 수행시 발생하는 상세한 정보들을 표준 출력으로 자세히 내보낼 것인지

Epoch 1/64
57/57 - 1s - loss: 444.6344 - val_loss: 70.0548 - 534ms/epoch - 9ms/step
Epoch 2/64
57/57 - 0s - loss: 76.1956 - val_loss: 56.1040 - 81ms/epoch - 1ms/step
Epoch 3/64
57/57 - 0s - loss: 76.0281 - val_loss: 50.2085 - 81ms/epoch - 1ms/step
Epoch 4/64
57/57 - 0s - loss: 64.8152 - val_loss: 44.8674 - 81ms/epoch - 1ms/step
Epoch 5/64
57/57 - 0s - loss: 55.6858 - val_loss: 36.4431 - 85ms/epoch - 1ms/step
Epoch 6/64
57/57 - 0s - loss: 56.0985 - val_loss: 35.2096 - 84ms/epoch - 1ms/step
Epoch 7/64
57/57 - 0s - loss: 48.4682 - val_loss: 23.3192 - 84ms/epoch - 1ms/step
Epoch 8/64
57/57 - 0s - loss: 49.7674 - val_loss: 21.9785 - 83ms/epoch - 1ms/step
Epoch 9/64
57/57 - 0s - loss: 47.2713 - val_loss: 25.4192 - 83ms/epoch - 1ms/step
Epoch 10/64
57/57 - 0s - loss: 37.1775 - val_loss: 37.7135 - 85ms/epoch - 1ms/step
Epoch 11/64
57/57 - 0s - loss: 42.4303 - val_loss: 22.7942 - 83ms/epoch - 1ms/step
Epoch 12/64
57/57 - 0s - loss: 51.4245 - val_loss: 21.9473 - 87ms/epoch - 2ms/step
Epoch 13/64

# 4. 평가 및 예측

In [5]:
loss = model.evaluate(x_test, y_test, verbose=2)
print('loss(mse): ', loss)

y_predict = model.predict(x_test)

RMSE = np.sqrt(mean_squared_error(y_test, y_predict))
print("RMSE: ", RMSE)

r2 = r2_score(y_test, y_predict)
print("R2: ", r2)

5/5 - 0s - loss: 13.9760 - 18ms/epoch - 4ms/step
loss(mse):  13.976022720336914
RMSE:  3.738451884350683
R2:  0.8059759223781373
